In [ ]:
import pandas as pd


# ==============================
# Вспомогательные функции
# ==============================

def td_to_hms(td):
    """Функция замены формата timedelta64 на hh:mm:ss без перехода в дни."""
    if pd.isna(td):
        return None  # защита от NaT / None
    try:
        total_seconds = int(td.total_seconds())
    except Exception:
        return None

    hours = total_seconds // 3600
    minutes = (total_seconds % 3600) // 60
    seconds = total_seconds % 60
    return f"{hours:02d}:{minutes:02d}:{seconds:02d}"


def clean_str(series):
    """Универсальная очистка строковых полей: приведение к str и trim."""
    return series.astype(str).str.strip()


def drop_empty_time_rows(df, cols):
    """Удаление строк, если в указанных полях пустые значения (пустая строка / пробелы / NaN)."""
    df[cols] = df[cols].replace(r'^\s*$', pd.NA, regex=True)
    return df.dropna(subset=cols)


def drop_zero_time_rows(df, cols):
    """Удаление строк, если время в указанных полях равно 00:00:00."""
    for col in cols:
        t = pd.to_datetime(
            df[col].astype(str).str.strip().str.replace('.', ':', regex=False),
            format="%H:%M:%S",
            errors="coerce"
        )
        mask_midnight = (t.dt.hour == 0) & (t.dt.minute == 0) & (t.dt.second == 0)
        df = df.loc[~mask_midnight].copy()
    return df


def apply_exceptions(df, sheet_name, target_col, sprav_file="spravochnik.xlsx"):
    """
    Удаление строк по справочнику исключений.
    Удалённые строки сохраняются в exception.xlsx.
    Поведение такое же, как в исходном коде: exception.xlsx перезаписывается при каждом вызове.
    """
    df_ref = pd.read_excel(sprav_file, sheet_name=sheet_name)
    df[target_col] = clean_str(df[target_col])
    df_ref["exception"] = clean_str(df_ref["exception"])

    match_values = set(df_ref["exception"].unique())
    mask = df[target_col].isin(match_values)

    df_exceptions_removed = df[mask].copy()
    df_exceptions_removed.to_excel("exception.xlsx", index=False)

    return df[~mask].copy()


def normalize_tab(series):
    """Нормализация табельного номера: к числу, затем к строке без пробелов."""
    s_num = pd.to_numeric(series, errors='coerce')
    s_int = s_num.astype('Int64')
    return s_int.astype(str).str.strip()


# ==============================
# ЧАСТЬ 1: РАБОТА С ФАЙЛОМ ASARV_NEW
# ==============================

print('ЧАСТЬ 1: РАБОТА С ФАЙЛОМ ASARV_NEW')
print("Преобразование данных")

df = pd.read_excel("asarv.xlsx")
df.columns = df.columns.str.replace(r'[\r\n]+', '', regex=True).str.strip()

# -- заполнение пустых ячеек в полях
for col in ['Подразделение', 'Сотрудник', 'Имя', 'Должность']:
    df[col] = df[col].ffill()
print('-- заполнение пустых ячеек в полях.')

# -- удаление строк с пустым и нулевым временем
time_cols = ['Время входа', 'Время выхода', 'Норма', 'Отработанное время']
df = drop_empty_time_rows(df, time_cols)
df = drop_zero_time_rows(df, time_cols)
print('-- удаление строк если время входа, выхода, норма, отр. время - нулевое или пустое.')

# (Блок "удаление строк с отрицательным временем" убран как мёртвый:
# после to_datetime отрицательного времени суток не бывает — на функциональность это не влияет.)

# ==============================
# СП / СП2 / СП3
# ==============================

# --- добавление полного названия СП
df_ref_sp = pd.read_excel("spravochnik.xlsx", sheet_name='SP')
df['Подразделение'] = clean_str(df['Подразделение'])
df_ref_sp['short_name'] = clean_str(df_ref_sp['short_name'])
sp_map = df_ref_sp.set_index('short_name')['full_name']
df['Полное наименование'] = df['Подразделение'].map(sp_map)
print('-- добавление полного названия СП.')

# --- добавление СП2 и СП3
df_ref = pd.read_excel("shtatka_go.xlsx")

df["Сотрудник"] = normalize_tab(df["Сотрудник"])
df_ref["ТабНомер"] = normalize_tab(df_ref["ТабНомер"])

df["Календарный день"] = pd.to_datetime(df["Календарный день"], errors="coerce", dayfirst=True)
df_ref["ДатаПриема"] = pd.to_datetime(df_ref["ДатаПриема"], errors="coerce", dayfirst=True)
df_ref["ДатаУвольнения"] = pd.to_datetime(df_ref["ДатаУвольнения"], errors="coerce", dayfirst=True)

df_ref["Подразделение 2"] = clean_str(df_ref["Подразделение 2"])
df_ref['Подразделение 3'] = clean_str(df_ref['Подразделение 3'])

# Если "Дата увольнения" пустая, считаем, что сотрудник всё ещё работает
open_end_date = pd.Timestamp("2099-12-31")
df_ref["ДатаУвольнения"] = df_ref["ДатаУвольнения"].fillna(open_end_date)

df = df.reset_index().rename(columns={"index": "row_id"})
ref_keep = ["ТабНомер", "ДатаПриема", "ДатаУвольнения", "Подразделение 2", "Подразделение 3"]
df_ref = df_ref[ref_keep].copy()

merged = df.merge(
    df_ref,
    how="left",
    left_on="Сотрудник",
    right_on="ТабНомер",
    suffixes=("", "_ref"),
)

in_range = (
    (merged["Календарный день"] >= merged["ДатаПриема"]) &
    (merged["Календарный день"] <= merged["ДатаУвольнения"])
)
candidates = merged[in_range].copy()
candidates = candidates.sort_values(["row_id", "ДатаПриема"])
best_match = candidates.drop_duplicates(subset="row_id", keep="last")

df = df.set_index("row_id").copy()
df["Подразделение 2"] = pd.NA
df["Подразделение 3"] = pd.NA

df.loc[
    best_match["row_id"].values,
    ["Подразделение 2", "Подразделение 3"]
] = best_match.set_index("row_id")[["Подразделение 2", "Подразделение 3"]].values

df = df.reset_index(drop=True)
print('-- добавление СП2 и СП3.')

# ==============================
# НОРМА, EXCEPTIONS, ПРИЗНАКИ
# ==============================

# --- нормализация Нормы в формат чч:мм:сс без перехода в дни
norma_clean = (
    df["Норма"].astype(str)
    .str.strip()
    .str.replace('.', ':', regex=False)
)
# Приводим вида 1:00:00 к 01:00:00
norma_clean = norma_clean.str.replace(r'^(\d):', r'0\1:', regex=True)

norma_td = pd.to_timedelta(norma_clean, errors="coerce")
df["Норма"] = norma_td.dt.components.apply(
    lambda x: f"{int(x['hours']):02d}:{int(x['minutes']):02d}:{int(x['seconds']):02d}",
    axis=1
)

# --- удаление по exceptions_ID, exceptions_roles, exceptions_sp
df = apply_exceptions(df, sheet_name='exceptions_ID', target_col='Сотрудник')
df = apply_exceptions(df, sheet_name='exceptions_roles', target_col='Должность')
df = apply_exceptions(df, sheet_name='exceptions_sp', target_col='Полное наименование')
print('-- удаление exceptions_ID, exceptions_roles, exceptions_sp.')

# --- признак вн.совместитель
df_ref = pd.read_excel("spravochnik.xlsx", sheet_name='sovmestitely')
df['Сотрудник'] = clean_str(df['Сотрудник'])
df_ref['ТабНомер'] = clean_str(df_ref['ТабНомер'])
df['Флаг совместитель'] = df['Сотрудник'].isin(df_ref['ТабНомер']).astype(int)
print('-- добавление признака вн.совместитель.')

# --- признак СУРВ
df_ref = pd.read_excel("spravochnik.xlsx", sheet_name='SURV')
df['Сотрудник'] = clean_str(df['Сотрудник'])
df_ref['ТабНомер'] = clean_str(df_ref['ТабНомер'])
df['Флаг СУРВ'] = df['Сотрудник'].isin(df_ref['ТабНомер']).astype(int)
print('-- добавление признака СУРВ.')

# --- признак категория роли
df_ref = pd.read_excel("spravochnik.xlsx", sheet_name='category')
df_ref['role'] = clean_str(df_ref['role'])
df['Должность'] = clean_str(df['Должность'])
role_map = df_ref.set_index('role')['category']
df['Категория должности'] = df['Должность'].map(role_map)
print('-- добавление признака категории должности.')

# --- флаг праздника и флаг "норма < 8"
df_ref = pd.read_excel("spravochnik.xlsx", sheet_name='spec_date')
df_ref["date-1"] = pd.to_datetime(df_ref["date-1"], errors="coerce")
holiday_set = set(df_ref["date-1"].dropna().unique())

df["Флаг праздник"] = df["Календарный день"].isin(holiday_set).astype(int)

norma_str = df['Норма'].astype(str).str.strip()
time_parts = norma_str.str.extract(r'^\s*(\d{1,2}):(\d{1,2}):(\d{1,2})\s*$')
time_parts = time_parts.fillna(0).astype(int)

hours = time_parts[0]
minutes = time_parts[1]
seconds = time_parts[2]
total_seconds = hours * 3600 + minutes * 60 + seconds

df['Флаг норма<8'] = (total_seconds < 8 * 3600).astype(int)
print('-- добавление признака Флаг норма < 8.')

# --- признак ЕСЦ
df["Полное наименование"] = clean_str(df["Полное наименование"])
df["ЕСЦ"] = df["Полное наименование"].str.contains("сервисный центр", case=False, na=False).astype(int)
print('-- добавление признака ЕСЦ.')

# --- признак КЦ
df["Подразделение 2"] = clean_str(df["Подразделение 2"])
df["КЦ"] = df["Подразделение 2"].str.contains("контакт-центр", case=False, na=False).astype(int)
print('-- добавление признака КЦ.')

# ==============================
# НОВОЕ ВРЕМЯ ВХОДА/ВЫХОДА, НОВАЯ НОРМА, ОБЕД
# ==============================

# --- новое время прихода/выхода (округление до часа с учётом 30 минут)
df["Время входа"] = pd.to_datetime(df["Время входа"], format="%H:%M:%S", errors="coerce")
df["Время выхода"] = pd.to_datetime(df["Время выхода"], format="%H:%M:%S", errors="coerce")

df["Новое время входа"] = (df["Время входа"] + pd.Timedelta(minutes=30)).dt.floor("H")
df["Новое время выхода"] = (df["Время выхода"] + pd.Timedelta(minutes=30)).dt.floor("H")

df["Новое время входа"] = df["Новое время входа"].dt.strftime("%H:%M:%S")
df["Новое время выхода"] = df["Новое время выхода"].dt.strftime("%H:%M:%S")
print("-- добавление поля новое время прихода, новое время выхода")

# --- новые поля: Новая норма, День недели
df['Новая норма'] = pd.to_datetime(df['Норма'], errors='coerce').dt.strftime('%H:%M:%S')
df['День недели'] = pd.to_datetime(df['Календарный день'], errors='coerce').dt.day_name()

weekday_map = {
    'Monday': 'Понедельник',
    'Tuesday': 'Вторник',
    'Wednesday': 'Среда',
    'Thursday': 'Четверг',
    'Friday': 'Пятница',
    'Saturday': 'Суббота',
    'Sunday': 'Воскресенье',
}
df['День недели'] = df['День недели'].map(weekday_map)
print('-- создание новых полей.')

# --- изменение новой нормы в пн-чт
df['Новая норма'] = pd.to_datetime(df['Новая норма'], errors='coerce')
df['Флаг СУРВ'] = pd.to_numeric(df['Флаг СУРВ'], errors='coerce')

weekdays_mon_to_thu = ['Понедельник', 'Вторник', 'Среда', 'Четверг']
mask_01 = (
    df['День недели'].isin(weekdays_mon_to_thu) &
    (df['Флаг СУРВ'] == 0) &
    (df['Флаг норма<8'] == 0)
)
df.loc[mask_01, 'Новая норма'] = (
    df.loc[mask_01, 'Новая норма'].dt.normalize() +
    pd.Timedelta(hours=8, minutes=15, seconds=0)
)
df['Новая норма'] = df['Новая норма'].dt.strftime('%H:%M:%S')
print('-- изменение новой нормы в пн-чт, сохранение нормы по СУРВ и нормы < 8.')

# --- изменение новой нормы в пт
df['Новая норма'] = pd.to_datetime(df['Новая норма'], errors='coerce')
df['Флаг СУРВ'] = pd.to_numeric(df['Флаг СУРВ'], errors='coerce')

weekdays_friday = ['Пятница']
mask_02 = (
    df['День недели'].isin(weekdays_friday) &
    (df['Флаг СУРВ'] == 0) &
    (df['Флаг норма<8'] == 0)
)
df.loc[mask_02, 'Новая норма'] = (
    df.loc[mask_02, 'Новая норма'].dt.normalize() +
    pd.Timedelta(hours=6, minutes=45, seconds=0)
)
df['Новая норма'] = df['Новая норма'].dt.strftime('%H:%M:%S')
print('-- изменение новой нормы в пт, сохранение нормы по СУРВ и нормы < 8.')

# --- добавление поля Время обеда
df["Время обеда"] = pd.NA
mask_obed_empty = df['Время обеда'].isna() | (df['Время обеда'].astype(str).str.strip() == '')
time_M = pd.to_timedelta(df['Новая норма'].astype(str), errors='coerce')

# Если время пусто и СУРВ, то 1 час
mask_obed1 = mask_obed_empty & (df['Флаг СУРВ'] == 1)
df.loc[mask_obed1, 'Время обеда'] = '01:00:00'
mask_obed_empty = df['Время обеда'].isna() | (df['Время обеда'].astype(str).str.strip() == '')

# Если время пусто и норма 8.15, то 45 мин
mask_obed2 = mask_obed_empty & (time_M == pd.Timedelta(hours=8, minutes=15))
df.loc[mask_obed2, 'Время обеда'] = '00:45:00'
mask_obed_empty = df['Время обеда'].isna() | (df['Время обеда'].astype(str).str.strip() == '')

# Если время пусто и норма 6.45, то 45 мин
mask_obed3 = mask_obed_empty & (time_M == pd.Timedelta(hours=6, minutes=45))
df.loc[mask_obed3, 'Время обеда'] = '00:45:00'
mask_obed_empty = df['Время обеда'].isna() | (df['Время обеда'].astype(str).str.strip() == '')

# Если время пусто и норма между 5.01 и 5.44, то 1 час
mask_obed4 = (
    mask_obed_empty &
    (time_M >= pd.Timedelta(hours=5, minutes=1)) &
    (time_M <= pd.Timedelta(hours=5, minutes=44))
)
df.loc[mask_obed4, 'Время обеда'] = '01:00:00'
mask_obed_empty = df['Время обеда'].isna() | (df['Время обеда'].astype(str).str.strip() == '')

# Если время пусто и норма между 6.46 и 8.14, то 1 час
mask_obed5 = (
    mask_obed_empty &
    (time_M >= pd.Timedelta(hours=6, minutes=46)) &
    (time_M <= pd.Timedelta(hours=8, minutes=14))
)
df.loc[mask_obed5, 'Время обеда'] = '01:00:00'
mask_obed_empty = df['Время обеда'].isna() | (df['Время обеда'].astype(str).str.strip() == '')

# Для остальных 0 часов
df.loc[mask_obed_empty, 'Время обеда'] = '00:00:00'
print('-- добавление поле время обеда.')

# ==============================
# РАСЧЕТ ИТОГОВЫХ ПОЛЕЙ
# ==============================

print("Расчеты итоговых полей")

# приведение ключевых временных полей к timedelta/datetime
df["Время входа"] = pd.to_datetime(df["Время входа"], errors="coerce")
df["Время выхода"] = pd.to_datetime(df["Время выхода"], errors="coerce")

for col in ["Отработанное время", "Время обеда", "Время отсутствия"]:
    df[col] = pd.to_timedelta(df[col].astype(str), errors="coerce")

# --- расчет нового отработанного времени
df["Новое отработанное время"] = pd.NaT

# 1. Если Отработанное время >= 24 часов → просто копируем
mask_ge_24 = df["Отработанное время"] >= pd.Timedelta(hours=24)
df.loc[mask_ge_24, "Новое отработанное время"] = df.loc[mask_ge_24, "Отработанное время"]

# общая маска для остальных
mask_other = ~mask_ge_24

# 2. Если время выхода < времени входа → сотрудник вышел на следующий день
mask_next_day = mask_other & (df["Время выхода"] < df["Время входа"])
df.loc[mask_next_day, "Новое отработанное время"] = (
    pd.Timedelta(hours=24)
    - (df.loc[mask_next_day, "Время входа"] - df.loc[mask_next_day, "Время входа"].dt.normalize())
    + (df.loc[mask_next_day, "Время выхода"] - df.loc[mask_next_day, "Время выхода"].dt.normalize())
)

# 3. Остальные строки → обычный расчет выход - вход
mask_normal = mask_other & ~mask_next_day
raw_work = df["Время выхода"] - df["Время входа"]
df.loc[mask_normal, "Новое отработанное время"] = raw_work[mask_normal]

# Разделяем на <5 часов и >=5 часов
mask_lt_5 = mask_normal & (raw_work < pd.Timedelta(hours=5))
mask_ge_5 = mask_normal & (raw_work >= pd.Timedelta(hours=5))

# Если < 5 часов → вычитаем только "Время отсутствия"
df.loc[mask_lt_5, "Новое отработанное время"] = (
    raw_work[mask_lt_5] - df.loc[mask_lt_5, "Время отсутствия"]
)

# Если ≥ 5 часов → вычитаем max(обед, отсутствие)
max_break = df[["Время обеда", "Время отсутствия"]].max(axis=1)
df.loc[mask_ge_5, "Новое отработанное время"] = (
    raw_work[mask_ge_5] - max_break[mask_ge_5]
)
print('-- расчет поля новое отработанное время.')

# --- расчет новой нормы с учетом праздников
df["Новая норма"] = pd.to_timedelta(df["Новая норма"], errors="coerce")
mask_hol = (df["Флаг праздник"] == 1) & (df["Флаг СУРВ"] == 0)
df.loc[mask_hol, "Новая норма"] = df.loc[mask_hol, "Новая норма"] - pd.Timedelta(hours=1)
print('-- расчет новой нормы с учетом праздников.')

# --- расчет поля новые переработки
df["Новое отработанное время"] = pd.to_timedelta(df["Новое отработанное время"], errors="coerce")

df["Новые переработки"] = df["Новое отработанное время"] - df["Новая норма"]
df.loc[
    (df["Новые переработки"].dt.total_seconds() <= 0) |
    (df["Новые переработки"].isna()),
    "Новые переработки"
] = pd.Timedelta(0)
print('-- расчет поля новые переработки.')

# --- расчет поля новые недоработки
df["Отработанное время"] = pd.to_timedelta(df["Отработанное время"].astype(str), errors="coerce")
df["Новые недоработки"] = df["Новая норма"] - df["Отработанное время"]
df.loc[
    (df["Новые недоработки"].dt.total_seconds() <= 0) |
    (df["Новые недоработки"].isna()),
    "Новые недоработки"
] = pd.Timedelta(0)
print('-- расчет поля новые недоработки.')

# --- перевод полей в формат чч:мм:сс перед выгрузкой
for col in [
    "Отработанное время",
    "Новая норма",
    "Новые недоработки",
    "Новые переработки",
    "Новое отработанное время",
    "Время обеда",
]:
    df[col] = df[col].apply(td_to_hms)
print('-- корректировка формата полей перед выгрузкой.')

# ==============================
# АГРЕГАЦИЯ ПЕРЕРАБОТОК И НЕДОРАБОТОК
# ==============================

# Переводим обратно из строкового hh:mm:ss в Timedelta для суммирования
for col in ["Новые переработки", "Новые недоработки"]:
    df[col] = pd.to_timedelta(df[col])

# Группируем по Имени и считаем сумму по двум полям
agg = df.groupby("Имя", as_index=False)[["Новые переработки", "Новые недоработки"]].sum()

# Переводим суммы в формат hh:mm:ss без перехода в дни (25:00:00 и т.п. сохраняются)
for col in ["Новые переработки", "Новые недоработки"]:
    agg[col] = agg[col].apply(td_to_hms)
    df[col] = df[col].apply(td_to_hms)

print('-- агрегация переработок и недоработок.')

# ==============================
# ВЫГРУЗКА
# ==============================

with pd.ExcelWriter("asarv_new.xlsx") as writer:
    df.to_excel(writer, sheet_name="asarv_new", index=False)
    agg.to_excel(writer, sheet_name="агрегация", index=False)

print('_______________________________________________________')
print('завершена работа с asarv_new')
print('')
